In [6]:
%matplotlib inline
import matplotlib.pyplot as plt
import collections
import gzip
import numpy as np
import os
import time
import umap

import pandas as pd

from collections import defaultdict
import itertools

# Interactive HTML tools
from ipywidgets import interact
import bokeh
import bokeh.io
from bokeh.io import push_notebook
from bokeh.plotting import figure, show, save, output_notebook, output_file
from bokeh.palettes import Category20b
from bokeh.palettes import Category20
from bokeh.palettes import Category10
from bokeh.palettes import PRGn
from bokeh.palettes import Set1

# Machine-learning and dimensionality reduction tools
import sklearn
from sklearn.decomposition import PCA as PCA # We'll use this to check our implementation
from sklearn.manifold import TSNE
from MulticoreTSNE import MulticoreTSNE as mTSNE

In [21]:
import plotly.plotly as py
import plotly.graph_objs as go

import plotly
from plotly.graph_objs import Scatter, Layout

# Import data and project

In [3]:
data_dir = '/Volumes/Stockage/alex/popres/Novembre_etal_2008_misc/files'

In [4]:
# Define the files we'll be using
pc_file = 'POPRES_08_24_01.EuroThinFinal.LD_0.8.exLD.out0-PCA.eigs'

#aux_path = os.path.join(hrs_data_dir, aux_file)
pc_path = os.path.join(data_dir, pc_file)

# Import PC data. This data must be converted to an array.
with open(pc_path) as pc:
    pca_contents = pc.readlines()

pca_data = []

for pc in pca_contents[1:]:
    pca_data.append(pc.split()[2:len(pc.split())-1])
    
# Purely numeric values of PCA
pca_data_array = np.array(pca_data).astype(np.float)

In [5]:
pca_data_array.shape

(1387, 20)

In [136]:
# Create projections for 2 to 20 PCs for varying perplexities
proj_dir = '/Users/alex/Documents/Ethnicity/POPRES/POPRES_projections'
pc_list = [i for i in range(2,21)]
plex_list = [1,2,10,30,100]
mtsne_plex_popres = []

for pl in plex_list:
    temp_list = []
    for pc in pc_list:
        temp_proj = mTSNE(n_components=2,n_jobs=4,perplexity=pl).fit_transform(pca_data_array[:,:pc])
        
        fname = 'mTSNE_POPRES_PC'+ str(pc) + '_PLEX' + str(pl)
        np.savetxt(os.path.join(proj_dir,fname), temp_proj)
        
        temp_list.append(temp_proj)
        print('Completed projection for PC dimension: ' + str(pc) + ' on perplexity ' + str(pl))
    
    mtsne_plex_popres.append(temp_list)

Completed projection for PC dimension: 2 on perplexity 1
Completed projection for PC dimension: 3 on perplexity 1
Completed projection for PC dimension: 4 on perplexity 1
Completed projection for PC dimension: 5 on perplexity 1
Completed projection for PC dimension: 6 on perplexity 1
Completed projection for PC dimension: 7 on perplexity 1
Completed projection for PC dimension: 8 on perplexity 1
Completed projection for PC dimension: 9 on perplexity 1
Completed projection for PC dimension: 10 on perplexity 1
Completed projection for PC dimension: 11 on perplexity 1
Completed projection for PC dimension: 12 on perplexity 1
Completed projection for PC dimension: 13 on perplexity 1
Completed projection for PC dimension: 14 on perplexity 1
Completed projection for PC dimension: 15 on perplexity 1
Completed projection for PC dimension: 16 on perplexity 1
Completed projection for PC dimension: 17 on perplexity 1
Completed projection for PC dimension: 18 on perplexity 1
Completed projection f

In [ ]:
pc_list = [i for i in range(2,21)]
mtsne_popres = []

for pc in pc_list:
    mtsne_popres.append(mTSNE(n_components=2,n_jobs=4).fit_transform(pca_data_array[:,:pc]))
    print('Completed projection for PC dimension: ' + str(pc))

In [33]:
# UMAP
proj_dir = '/Volumes/Stockage/alex/popres/projections'
tstamp_log = ''.join([str(t) for t in time.gmtime()[0:6]])
pc_list = [i for i in range(2,31)]

nc = 2
nn = 5
md = 0.5

for pc in pc_list:
    temp_proj = umap.UMAP(n_components=nc, n_neighbors=nn, min_dist=md).fit_transform(pca_data_array[:,:pc])
    fname ='POPRES_UMAP_PC'+str(pc)+'_NC'+str(nc)+'_NN'+str(nn)+'_MD'+str(md)+'_'+tstamp_log
    np.savetxt(os.path.join(proj_dir,fname), temp_proj)

In [34]:
# UMAP
proj_dir = '/Volumes/Stockage/alex/popres/projections'
tstamp_log = ''.join([str(t) for t in time.gmtime()[0:6]])
pc_list = [i for i in range(2,31)]

nc = 2
nn = 5
md = 0.1

for pc in pc_list:
    temp_proj = umap.UMAP(n_components=nc, n_neighbors=nn, min_dist=md).fit_transform(pca_data_array[:,:pc])
    fname ='POPRES_UMAP_PC'+str(pc)+'_NC'+str(nc)+'_NN'+str(nn)+'_MD'+str(md)+'_'+tstamp_log
    np.savetxt(os.path.join(proj_dir,fname), temp_proj)

# Auxiliary data and colouring

In [12]:
# Auxiliary data is split into two files: one containing ID to colour, and one containing colour to ethnicity
color_file = 'POPRESID_Color.txt'
country_color_file = 'ColorTablePCmap.txt'
label_file = 'PCA.txt'

color_file_list = []
country_color_file_list = []
color_translator = {}
pop_labels = []

with open(os.path.join(data_dir, color_file)) as colors:
    color_contents = colors.readlines()
    
for c in color_contents:
    color_file_list.append(c.strip().split())

with open(os.path.join(data_dir, country_color_file)) as country:
    country_contents = country.readlines()

for c in country_contents:
    country_color_file_list.append(c.strip().split('\t'))
    
# The colour list is for R so I've created a version with hex/RGB values
# Source: https://www.webucator.com/blog/2015/03/python-color-constants-module/
# I changed some of the colour names because they didn't exist
with open('/Users/alex/Documents/Ethnicity/POPRES/popres_colours') as colors:
    color_contents = colors.readlines()
    
for c in color_contents:
    c_name = c.strip().split()[0]
    c_hex = c.strip().split()[1]
    color_translator.update({c_name:c_hex})
    #color_translator_list.append(c.strip().split()[0:2])
    
# Population labels are stored in the PCA files
with open(os.path.join(data_dir,label_file)) as labels:
    label_contents = labels.readlines()
    
for l in label_contents[1:]:
    pop_labels.append(l.strip().split('\t')[0:2])

In [13]:
# Convert the to pandas dataframes to make the joins a bit easier.
popres_color_df = pd.DataFrame.from_records(color_file_list)
popres_country_df = pd.DataFrame.from_records(country_color_file_list)
popres_labels_df = pd.DataFrame.from_records(pop_labels)

popres_color_df.columns = ['ID','Colour']
popres_country_df.columns = ['Country','Colour']
popres_labels_df.columns = ['ID','Country']

In [14]:
# Combine the files
popres_combined_df = popres_labels_df.merge(popres_color_df, left_on='ID', right_on='ID', how='left' )

In [15]:
popres_combined_df

,ID,Country,Colour
0,474,Croatia,dodgerblue3
1,660,Serbia and Montenegro,cadetblue2
2,1890,Czech Republic,yellowgreen
3,1923,Bosnia and Herzegovina,cadetblue
4,2083,Serbia and Montenegro,cadetblue2
5,2733,Serbia and Montenegro,cadetblue2
6,3467,Serbia and Montenegro,cadetblue2
7,3548,Romania,darkseagreen4
8,3740,Serbia and Montenegro,cadetblue2
9,4108,Bosnia and Herzegovina,cadetblue


In [16]:
# Dictionary of countries to colours
country_color_dict = dict(popres_country_df.values.tolist())

In [17]:
# Create a dict of indices by country
indices_of_population_members_popres = defaultdict(list)

idx = 0
for p in popres_combined_df.values.tolist():
    try:
        indices_of_population_members_popres[p[1]].append(idx)
    except KeyError:
        continue
        
    idx+=1

In [ ]:
countries = popres_country_df.Country.values.tolist()

# 3D plotting

In [29]:
proj_path='//Volumes/Stockage/alex/popres/projections/POPRES_UMAP_PC10_NC3_NN5_MD0.5_201842934123'
popres_umap_proj=np.loadtxt(proj_path)

temp_size=5
popres_traces=[]

for country in countries:
    temp_proj = popres_umap_proj[indices_of_population_members_popres[country],:]
    color_string = str(int('0x' + color_translator[country_color_dict[country]][1:3].upper(),16)) + ',' + \
    str(int('0x' + color_translator[country_color_dict[country]][3:5].upper(),16)) + ',' + \
    str(int('0x' + color_translator[country_color_dict[country]][5:7].upper(),16))
    temp_trace = go.Scatter3d(
        x = temp_proj[:,0],
        y = temp_proj[:,1],
        z = temp_proj[:,2],
        name=country,
        mode='markers',
        marker=dict(
            size=temp_size,
            color='rgba('+color_string+',0.3)',
            line=dict(
                color='rgb('+color_string+')',
                width=2
            )
        )
    )
    popres_traces.append(temp_trace)

In [30]:
layout = go.Layout(
    margin=dict(
        l=0,
        r=0,
        b=0,
        t=0
    )
)

fig = go.Figure(data=popres_traces, layout=layout)
plotly.offline.plot(fig, filename='/Volumes/Stockage/alex/popres/3d/test_3d.html')

'file:///Volumes/Stockage/alex/popres/3d/test_3d.html'

# Plotting

In [36]:
# First replicate - this one used default perplexity
proj_dir = '/Volumes/Stockage/alex/popres/projections'
out_dir = '/Volumes/Stockage/alex/popres/2d'

for fname in os.listdir(proj_dir):
    if 'NC2' in fname:
        cur_proj = np.loadtxt(os.path.join(proj_dir,fname))

        p = figure(plot_width=1500, plot_height=1200)
        p.title.text = 'UMAP'

        for country in countries:
            proj_pop = cur_proj[indices_of_population_members_popres[country]]
            p.circle(proj_pop[:,0], proj_pop[:,1], legend=country, color=color_translator[country_color_dict[country]])

        p.legend.location = "top_left"

        p.legend.click_policy="hide"

        output_file(os.path.join(out_dir,fname+'.html'),title='UMAP')

        save(p)

In [149]:
# Second replicate - this one used several perplexities
out_dir = '/Users/alex/Documents/Ethnicity/POPRES/POPRES_HTML/second_replicate'
proj_dir = '/Users/alex/Documents/Ethnicity/POPRES/POPRES_projections'

# Data is contained in the list mtsne_plex_popres, which is indexed first by perplexity and second by PCs
for file in os.listdir(proj_dir):
    pc = file.split('PC')[1].split('_')[0]
    pl = file.split('PLEX')[1].split('_')[0]

    cur_proj = np.loadtxt(os.path.join(proj_dir,file))

    p = figure(plot_width=1500, plot_height=1200)
    p.title.text = 't-SNE with plex ' + str(pl) + ' on ' + str(pc) + ' components'

    for country in countries:
        proj_pop = cur_proj[indices_of_population_members_popres[country]]
        p.circle(proj_pop[:,0], proj_pop[:,1], legend=country, color=color_translator[country_color_dict[country]])

    p.legend.location = "top_left"

    p.legend.click_policy="hide"

    output_file(out_dir + '/POPRES_TSNE_PLEX' + str(pl) + '_PC' + str(pc) + '.html',title='t-SNE on ' + str(pc) + ' components')

    save(p)
    


In [ ]:
TSNE(n_iter=)